In [ ]:
# Install libraries
!pip install pandas

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

In [ ]:
# Import the raw file
# df = pd.read_csv(r'raw-files/street-trees.csv')
df = pd.read_csv(r'street-trees.csv')


# Select columns
df = df[['_id', 'WARD', 'COMMON_NAME', 'geometry']]

# Filter 'COMMON_NAME' column: Keep the name before comma
df['COMMON_NAME'] = df['COMMON_NAME'].apply(lambda x: str(x).split(',', 1)[0])

# Seperate latitude and longitude from 'geometry'
df['geometry'] = df['geometry'].apply(lambda x: str(x).split(',', 1)[1])
df['geometry'] = df['geometry'].apply(lambda x: str(x).split('(', 1)[1])
df['geometry'] = df['geometry'].apply(lambda x: str(x).split(')', 1)[0])
df['latitude'] = df['geometry'].apply(lambda x: str(x).split(',', 1)[0])
df['longitude'] = df['geometry'].apply(lambda x: str(x).split(',', 1)[1])

# Select columns
df = df[['_id', 'WARD', 'COMMON_NAME', 'latitude', 'longitude']]

In [ ]:
# Group wards into community area
toronto = [10, 11, 12, 13, 14]

df['WARD'] = df['WARD'].replace([1, 2, 3, 5, 7], 'Etobicoke York')
df['WARD'] = df['WARD'].replace([6, 8, 15, 16, 17, 18], 'North York')
df['WARD'] = df['WARD'].replace([4, 9, 10, 11, 12, 13, 14, 19], 'Toronto and East York')
df['WARD'] = df['WARD'].replace([20, 21, 22, 23, 24, 25], 'Scarborough')

# Rename 'WARD'
df.rename(columns = {'WARD':'AREA'}, inplace = True)
df

,_id,AREA,COMMON_NAME,latitude,longitude
0,1,Toronto and East York,Ash,-79.4161873311,43.6495364522
1,2,North York,Birch,-79.3545349538,43.8037189559
2,3,Scarborough,Maple,-79.2760802498,43.6776251577
3,4,North York,Elm,-79.425205778,43.7436916068
4,5,North York,Spruce,-79.3153757934,43.7338889215
...,...,...,...,...,...
662147,662148,Toronto and East York,Linden,-79.4304989828,43.6621039259
662148,662149,Scarborough,Oak,-79.3235015348,43.8084725945
662149,662150,North York,Maple,-79.4716487615,43.755339238
662150,662151,Toronto and East York,Maple,-79.3254415253,43.6811517468


In [ ]:
# Export the dataframe to JSON
df.to_json('trees_geo.json', orient='records')

In [ ]:
# Counts by community area
df_count_area = df.groupby(['AREA'])['AREA'].count().reset_index(name="total")

# Print the data frame
df_count_area

In [ ]:
# Count the sum of each tree
df_count_tree = df.groupby(['COMMON_NAME'])['COMMON_NAME'].count().reset_index(name="COUNT")
df_count_tree['COMMON_NAME'] = df_count_tree['COMMON_NAME'].replace('nan', 'Unknown')
df_count_tree

In [ ]:
# Group by 'COMMON_NAME' and 'AREA'
df_tree_area = df.groupby(['COMMON_NAME', 'AREA']).size().reset_index(name="COUNT")
df_tree_area['COMMON_NAME'] = df_tree_area['COMMON_NAME'].replace('nan', 'Unknown')
df_tree_area

In [ ]:
# Create a new dataframe for JSON
list_common = df_tree_area['COMMON_NAME'].drop_duplicates().tolist()
df_json = pd.DataFrame(list_common, columns=['Common'])
df_json['Latin'] = list_common

In [ ]:
# Add the count of trees for each council area
list_council_areas = ['Etobicoke York', 'North York', 'Toronto and East York', 'Scarborough']

def countTreesByName(area, tree):
    count_trees_areas = df_tree_area.loc[(df_tree_area['COMMON_NAME'] == tree) & (df_tree_area['AREA'] == area)]

    if (count_trees_areas.empty == True):
        count_trees = 0
    else:
        count_trees = count_trees_areas['COUNT'].values[0]
    return count_trees

list_trees_etobicoke = [];
list_trees_north_york = [];
list_trees_toronto = [];
list_trees_scarborough = [];

def addCounts(area, li):
    for tree in list_common:
        count = countTreesByName(area, tree)
        li.append(str(count))
    df_json[area] = li

addCounts('Etobicoke York', list_trees_etobicoke)
addCounts('North York', list_trees_north_york)
addCounts('Toronto and East York', list_trees_toronto)
addCounts('Scarborough', list_trees_scarborough)

df_json

In [ ]:
# Export the dataframe to JSON
df_json.to_json('common.json', orient='records')